Testing the deep forest module on a part of the SVMA data

Instructions: https://deepforest.readthedocs.io/en/latest/getting_started.html#issues

Modifications:
- first run throws an error regarding incompatible torch/torchvision. Reinstalled after installing deep forest using the following command in anaconda prompt, correct environmnt:
uninstall torch torchvision && pip install torch torchvision --no-cache-dir

In [ ]:
from deepforest import main
from deepforest import get_data
from deepforest import utilities

import matplotlib.pyplot as plt
from pathlib import Path

model = main.deepforest()
model.use_release()


In [ ]:
def reverse_RGB(img):
    return img[:,:,::-1]

In [ ]:
import cv2
data_image = Path('../../data/Fieldwork_data/PNM/gpkg_borore/area_recortada_mosaico.tif')
image_whole = reverse_RGB(cv2.imread(data_image.as_posix()))

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(9.7, 7.6)
ax.imshow(image_whole)

In [ ]:
bbox1 = [4200,5000, 1200, 2000] # t, b, l, r, 
bbox2 = [5000,5800, 2000, 2800]
bbox3 = [1200, 2000, 3700, 4500]
bbox4 = [2000, 2800, 4500, 5300]
bbox5 = [700, 1500, 6200, 7000]
bbox6 = [700, 1500, 7000, 7800]
list_bbox = [bbox1, bbox2, bbox3, bbox4, bbox5, bbox6]

In [ ]:
import cv2

list_imgs = []
for bbox in list_bbox:
    img = image_whole[bbox[0]:bbox[1], bbox[2]:bbox[3], :]
    list_imgs.append(img)

In [ ]:
fig, axes = plt.subplots(2,3)
fig.set_size_inches(15, 9)
for img, ax in zip(list_imgs, axes.flatten()):
    ax.imshow(img)
    ax.set_aspect('equal', adjustable='box')
fig.tight_layout()

In [ ]:
# Bench mark prediction
fig, axes = plt.subplots(2,3)
fig.set_size_inches(15, 9)
for img, ax in zip(list_imgs, axes.flatten()):
    predicted_img = model.predict_image(image=img, return_plot=True, thickness=5, color=[150, 0, 230])
    ax.imshow(reverse_RGB(predicted_img))
    ax.set_aspect('equal', adjustable='box')
fig.savefig('plots/benchmark.png')

In [ ]:
patch_size_range = range(200,600,100)
patch_overlap=0.35

for patch_size in patch_size_range:
    list_imgs_predict = []

    fig, axes = plt.subplots(2,3)
    fig.set_size_inches(15, 9)
    for img, ax in zip(list_imgs, axes.flatten()):
        predicted_img = model.predict_tile(image=img, return_plot=True, thickness=5, color=[150, 0, 230], patch_size=patch_size,patch_overlap=patch_overlap)
        ax.imshow(reverse_RGB(predicted_img))
        ax.set_aspect('equal', adjustable='box')
    fig.savefig(f'plots/patchsize{patch_size}_overlap{patch_overlap}.png')

In [ ]:
import numpy as np
patch_size = 500
patch_overlap_range=np.arange(0.25,0.75,0.1)

for patch_overlap in patch_overlap_range:
    list_imgs_predict = []
    fig, axes = plt.subplots(2,3)
    fig.set_size_inches(20, 12)
    for img, ax in zip(list_imgs, axes.flatten()):
        predicted_img = model.predict_tile(image=img, return_plot=True, thickness=5, color=[150, 0, 230], patch_size=patch_size,patch_overlap=patch_overlap)
        ax.imshow(reverse_RGB(predicted_img))
        ax.set_aspect('equal', adjustable='box')
    fig.savefig(f'plots/patchsize{patch_size}_overlap{patch_overlap}.png')

In [ ]:
patch_size = 600
patch_overlap = 0.35
use_soft_nms_list = [True, False]


for flag_nms in use_soft_nms_list:
    list_imgs_predict = []
    fig, axes = plt.subplots(2,3)
    fig.set_size_inches(15, 9)
    for img, ax in zip(list_imgs, axes):
        predicted_img = model.predict_tile(
            image=img,
            return_plot=True,
            thickness=5,
            color=[150, 0, 230],
            patch_size=patch_size,
            patch_overlap=patch_overlap,
            use_soft_nms=flag_nms
        )
        ax.imshow(reverse_RGB(predicted_img))
        ax.set_aspect("equal", adjustable="box")
    fig.savefig(f"plots/patchsize{patch_size}_overlap{patch_overlap}_nms{str(flag_nms)}.png")